# Part A: Build a code understanding model. Upload your own custom code files to the model and ask questions based on the code file as context.


In [1]:
# ! pip install openai==0.28

In [19]:
import base64

encoded_key = "c2stcHJvai1taDlMZEZteThyV1g2OW04bjYyb1QzQmxia0ZKRUVwbjZYVnFhU3BKUVBvdkhHSVM="
api_key = base64.b64decode(encoded_key).decode('utf-8')


In [4]:
import os
import openai

print(openai.__version__)
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) 
openai.api_key = api_key

0.28.0


In [5]:
def ask_gpt(prompt, model="gpt-3.5-turbo"):
    """
    This method will make a API call to the OPEN-AI server and get the response.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

ask_gpt("who are you")

'I am a language model AI created by OpenAI. I am here to assist you with any questions or tasks you may have. How can I help you today?'

In [6]:
def ask_gpt_with_context(prompt, model="gpt-3.5-turbo", context=None):
    """
    This method talk to OPEN-AI server and also update the context with the
    communication and keep all the communication and context at one place, which
    can be later used for  queries.
    """

    messages = []

    if context:
        messages.extend(context)

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message["content"], messages

In [7]:
code = "https://raw.githubusercontent.com/anjali-ojha/customer_segmentation/main/scripts/data_ingestion.py"

questions = [
    "Load data from website - https://raw.githubusercontent.com/anjali-ojha/customer_segmentation/main/scripts/data_ingestion.py",
    "what language this code written",
    "what it do?"
]

In [8]:
# Running a sequence of the questions though the GPT server.
import time

import requests
from bs4 import BeautifulSoup

def extract_website_text(website_url):
    """ This method will read the webpage fromn the link and return the content of it."""
    response = requests.get(website_url)
    website_text = response.content
    return website_text




context = None

for question in questions:
    print(f"\nUser : {question}")
    question = question.strip()

    if question.lower().startswith("load"):

        extra_data = ""
        web_url = [x for x in question.split(" ") if x.startswith("https://")][0]
        # print("url = ", web_url)
        website_data = extract_website_text(web_url)
        extra_data = website_data[:5000]
        # print(f"{extra_data = }")
        question = f""" remember the data

        ```{extra_data}```
        """

    else:
        question = question

    # print(question)
    response, new_context = ask_gpt_with_context(question, context=context)
    new_context.append({"role": "assistant", "content" : response})
    context = new_context

    print(f"\nAssistant: {response}")
    print("\n------------------------------------\n")
    time.sleep(3)


User : Load data from website - https://raw.githubusercontent.com/anjali-ojha/customer_segmentation/main/scripts/data_ingestion.py

Assistant: e kafka topic.
        it also performs the transformation on the data, by converting the json string to the dataframe.
        """
        topicName = "reviews"
        stream_df = self.read_from_topic(spark, topicName)
        schema = StructType([
            StructField("business_id", StringType(), True),
            StructField("cool", LongType(), True),
            StructField("date", StringType(), True),
            StructField("funny", LongType(), True),
            StructField("review_id", StringType(), True),
            StructField("stars", DoubleType(), True),
            StructField("text", StringType(), True),
            StructField("useful", LongType(), True),
            StructField("user_id", StringType(), True),
        ])
        df_result = stream_df.select(from_json(col("json_string"), schema).alias("data"))
        self.w

# Part B: Write a chatbot prompt to iteratively create a sequence of chats on one particular custom data.
1. The chatbot should be able to answer the questions based on the text data or multiple documents.
2. The chatbot should save the conversation in the memory.
2. Summarize the chats at the end of the conversation.

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.0)

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)


/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
x = conversation.predict(input="Hi, my name is Anjali, and Who are you?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Hi, my name is Anjali, and Who are you?

AI    : Hello Anjali! I am an AI assistant designed to help answer your questions and provide information. My name is Assistant. How can I assist you today?



In [11]:
def extract_website_text(website_urls):
    """ This method will read the webpage fromn the link and return the content of it."""
    res = []
    for website_url in website_urls:
        response = requests.get(website_url)
        # website_text = response.content
        soup = BeautifulSoup(response.text, 'html.parser')
        website_text = '\n'.join([p.get_text() for p in soup.find_all('p')])
        res.append(website_text)
    
    return res



knowledge_base = extract_website_text(["https://medium.com/@jheltzer/travel-guide-yosemite-national-park-2e83c8d6c2e9", "https://medium.com/@alissaalterishea/mount-rainier-national-park-washington-fe29c7dc1510"])

source_knowledge = "\n".join(knowledge_base)

context = f"""Rememer this context given in ``` and answer all question using that when asked. Acknowledge once data remeberd. 

```{source_knowledge}```
"""


x = conversation.predict(input=context)
memory.chat_memory.messages[-1].content

'I have remembered the context provided. How can I assist you with the information from the context?'

In [12]:
conversation.predict(input="Where is Yosemite located?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Where is Yosemite located?

AI    : Yosemite National Park is located in northeast California, east of the fertile San Joaquin Valley and just west of the border with Nevada. It is 167 miles straight east of San Francisco and covers almost 1,200 square miles.



In [13]:
conversation.predict(input="What are top locations in Yosemite?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : What are top locations in Yosemite?

AI    : Some of the top locations in Yosemite National Park include Glacier Point, Tunnel View, Lower Yosemite Falls, El Capitan, Mariposa Grove, Olmsted Point, Half Dome, and the Valley Floor Loop Trail. These locations offer stunning views, waterfalls, rock formations, and opportunities for hiking and exploring the park.



In [14]:
conversation.predict(input="Where is Mount Rainer Park located?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Where is Mount Rainer Park located?

AI    : Mount Rainier National Park is located in Washington state, just a couple of hours from Seattle. It is home to the tallest peak in the Cascade Range and is known for its active volcano, Mount Rainier.



In [15]:
conversation.predict(input="Which is the famous trail for the mount rainer")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Which is the famous trail for the mount rainer

AI    : One of the famous trails in Mount Rainier National Park is the Skyline Trail, which offers rugged terrain, snow-capped peaks, glacier-clad slopes, fields of wildflowers, waterfalls, and stunning views. It is a steep 5.5 mile trek that is highly recommended for hikers and adventurers.



In [16]:
conversation.predict(input="Which park is better?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Which park is better?

AI    : As an AI, I don't have personal preferences. Both Yosemite National Park and Mount Rainier National Park are unique and beautiful in their own ways. Yosemite is known for its iconic rock formations, waterfalls, and diverse landscapes, while Mount Rainier offers stunning views of an active volcano, glaciers, and wildflower-filled meadows. It ultimately depends on your preferences and what you're looking to experience in a national park.



In [17]:
conversation.predict(input="Give me the whole conversation Summary?")
resp = memory.chat_memory.messages[-1].content
print(f"""
Human : {memory.chat_memory.messages[-2].content}

AI    : {resp}
""")


Human : Give me the whole conversation Summary?

AI    : The conversation covered information about Yosemite National Park and Mount Rainier National Park. Yosemite is located in northeast California and covers almost 1,200 square miles, with top locations including Glacier Point, Tunnel View, Lower Yosemite Falls, El Capitan, and more. Mount Rainier National Park is located in Washington state, near Seattle, and features the Skyline Trail as a famous hiking destination. Both parks offer unique and beautiful landscapes for visitors to explore.



In [18]:
# Create Conversation Summary
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


def create_summary(memory):
    llm = OpenAI(temperature=0, openai_api_key=openai.api_key)
    
    text = "\n".join([x.content for x in memory.chat_memory.messages])
    num_tokens = llm.get_num_tokens(text)
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=5000, chunk_overlap=350)
    docs = text_splitter.create_documents([text])

    chain = load_summarize_chain(llm=llm, chain_type='map_reduce') 
    return chain.run(docs)

summary = create_summary(memory)
print("\n\n".join(summary.replace("\\n", "").split(". ")))

/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(




The article provides tips and recommendations for visiting Yosemite National Park, including lodging options, transportation, and popular attractions

It also discusses the challenges and highlights of various hiking trails in the park

The writer also briefly mentions their visit to Mount Rainier National Park and their experience as a teacher, traveler, and outdoor enthusiast

Both parks offer unique landscapes and popular hiking trails for visitors to enjoy.
